# Introduction au package KERAS et au "Deep-Learning": réseaux de neurones convolutifs  (CNNs) & reconnaissance d'images

Dans ce notebook nous allons apprendre à manipuler les fonctions de base du [package Keras](https://keras.io/) permettant de mettre en oeuvre des algorithmes de réseaux de neurones, dits de "deep learning". Nous considérerons pour cela le problème le plus classique : catégorisation d'images par réseaux de neurones convolutifs (CNNs : Convolutional Neural Networks).
Nous travaillerons sur la base MNIST qui présente l'avantage (1) d'être de taille relativement restreinte, ce qui permet de réaliser l'analyse dans un temps raisonnable, et (2) d'être déjà disponible dans Keras, ce qui permet de gagner du temps sur des questions de mise en forme des données.

Le plan sera le suivant  :
1. **préparation des données**
2. **réseaux de neurones à une couche & régression (logistique) multinomiale**
3. **réseaux de neurones multicouches & modèles non-linéaires**
4. **réseaux de neurones convolutifs & apprentissage de représentation**

On commence par charger les packages et par bloquer la graine du générateur aléatoire par souci de reproductibilité. 
**Attention : keras faisant partie de tensorflow depuis la version 2 de tensorflow, les imports ne seront pas les mêmes si vous travaillez à partir de la version 2 ou d'une version antérieure.**
Pour cela, renseigner la variable **"tensorflow_version"** dans la cellule ci-dessous à la valeur **"V1"** ou **"V2"** en fonction de votre configuration.

In [ ]:
# specify tensorflow version
tensorflow_version = "V2"

In [ ]:
# import packages
import csv
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pickle
# import tensorflow requirements - depending on version
if tensorflow_version == "V1":
    from keras import backend as K
    from keras.models import Model
    from keras.models import Sequential
    from keras.models import load_model
    from keras.layers import Dense, Activation, Flatten
    from keras.layers import Conv2D, MaxPooling2D
    from keras.utils import to_categorical
if tensorflow_version == "V2":
    from tensorflow.keras.models import Model
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.models import load_model
    from tensorflow.keras.layers import Dense, Activation, Flatten
    from tensorflow.keras.layers import Conv2D, MaxPooling2D
    from tensorflow.keras.utils import to_categorical
# set random seed
np.random.seed(123)

# Partie 1. Préparation des données

Nous commençons par charger le jeu de données sauvegardé par le premier notebook.

In [ ]:
f = open('mnist-dataset.pickle', 'rb')
dataset = pickle.load(f)
f.close()

In [ ]:
X_train = dataset[0]
X_test  = dataset[1]
Y_train = dataset[2]
Y_test  = dataset[3]

# Partie 2. Réseaux de neurones à une couche & régression logistique multinomiale

Voir notebook précedent.
On se contente de charger le modèle sauvegardé.

In [ ]:
linreg = load_model("model_multinomial")

# Partie 3. Réseaux de neurones multicouches & modèles non-linéaires

Voir notebook précedent.
On se contente de charger le modèle sauvegardé.

In [ ]:
mlp = load_model("model_mlp")

# Partie 4. Réseaux de neurones convolutifs & apprentissage de représentation

Les architectures considérées jusqu'à présent n'ont pas exploité la structure naturelle des images car nous les avons transformées comme des vecteurs de descripteurs. La corrélation existante entre pixels proches est par conséquent perdue.

Les réseaux de neurones convolutifs (Convolutional Neural Networks) permettent de travailler directement sur les images, sans passer par une étape de transformation "manuelle" de descripteurs. 
Pour cela, ils commencent typiquement par alterner (principalement) des couches: 
* de convolution, appliquant des transformation locales sur les images,
* de "pooling", réduisant graduellement la dimension spatiale des images.

A l'issue de cette succession de transformations, une représentation vectorielle est obtenue en concaténant la sortie de la dernière couche dans un (grand) vecteur -- comme ce que nous avons fait précédemment -- qui est finalement passée en entrée à un perceptron (multi-couche ou pas). 
La succession des opérations de convolutions / pooling en amont du perceptron permet de construire des représentations haut niveau des images, bénéficiant par ailleurs de propriétés d'invariance (e.g., translation). Le point clé réside dans le fait que les poids des masques de convolution sont optimisés à partir des données: on parle parfois d'apprentissage de représentation.

<img src="pic/cnn.png" width="600">

Ce type d'architecture est très simple à implémenter dans Keras.
Avant cela, **il faut néanmoins appliquer une nouvelle transformation à nos données**.
En effet, dans ce type de modèle, toutes les données sont en trois dimensions : 2 dimensions spatiales, plus 1 troisième dimension qui correspond:
* au nombre de filtres de convolution de la couche précédente dans les couches internes
* au nombre de canaux des images dans la couche d'entrée

Ce nombre de canaux est typiquement égal à 3 pour des images couleur (les canaux RGB).
Pour une image en niveaux de gris (comme c'est le cas ici) on ne dispose que d'un canal : pour pouvoir les considérer dans Keras, il faut donc leur ajouter une 3ème dimension "virtuelle".
Les données d'entrée se présentent alors comme un tenseur à 4 dimensions : 
* #1: le nombre d'images
* #2 et #3 : les dimensions spatiales
* #4 : le nombre de canaux


In [ ]:
X_train_3d = np.reshape(X_train,[X_train.shape[0],X_train.shape[1],X_train.shape[2],1])
X_test_3d  = np.reshape(X_test,[X_test.shape[0],X_test.shape[1],X_test.shape[2],1])
X_train_3d.shape

Pour construire un CNN dans Keras, il suffit donc de commencer par ajouter une succesion de couches : 
* des couches de type **Conv2D** pour les convolutions
* des couches de type **MaxPooling2D** ou **AveragePooling2D** pour le pooling, résumant une zone de l'image à sa valeur maximum ou moyenne.

Ensuite :
1. on convertit la sortie de la dernière couche en un vecteur via une couche de type **Flatten**
2. on définit un perceptron (multi-couche) ou pas en ajoutant des couches de type **Dense** comme précédemment.

A noter que **chaque couche de convolution inclut une fonction d'activation**, qu'on définit généralement comme une fonction ReLu.

## <span style="color:red"> $\Rightarrow$ EXERCICE: en vous inspirant de l'exemple donné en cours, constuire un réseau mettant en jeu : </span>
* <span style="color:red">2 couches de convolutions impliquant chacune 32 filtres de dimension (spatiale) 3 x 3.</span>
* <span style="color:red">une couche de "max pooling" de dimension spatiale 2 x 2, réduisant la taille de l'image par 2.</span>
* <span style="color:red">un perceptron à une couche interne, mettant en jeu 128 neurones.</span>


In [ ]:
# TODO : implement CNN model
cnn = Sequential()


On peut visualiser la taille des sorties des différentes couches via la méthode *summary()*.
A noter qu'à l'issue des étapes des convolutions, les dimensions spatiales sont réduites. On peut faire en sorte de les préserver si besoin en spécifiant l'argument *padding='same'* lors des appels à la fonction *Conv2D()*. 

In [ ]:
cnn.summary()

Il ne reste qu'à compiler le modèle et l'apprendre. 
Notons que la procédure est plus longue que précédemment, ce qui n'est pas suprenant du fait du nombre de paramètres qui est bien plus grand. Ici, on a réduit le nombre d'itérations (epochs) à 3 pour limiter le temps de calcul.

In [ ]:
cnn.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy']) 
cnn.fit(X_train_3d, Y_train, batch_size=32, epochs=3, verbose=1, validation_split=0.2)

Enfin, on peut calculer les performances des différentes stratégies :

In [ ]:
# re-generate vectorized version to evaluate multinomial and mlp networks
input_dim = 28*28 
X_train_vec = X_train.reshape(60000, input_dim) 
X_test_vec  = X_test.reshape(10000, input_dim)

In [ ]:
score_linreg = linreg.evaluate(X_test_vec, Y_test, verbose=0)
score_mlp = mlp.evaluate(X_test_vec, Y_test, verbose=0)
score_cnn = cnn.evaluate(X_test_3d, Y_test, verbose=0)
print("multinomial regression test-set accuracy =", score_linreg[1])
print("multi-layer perceptron test-set accuracy =", score_mlp[1])
print("CNN test-set accuracy =", score_cnn[1])

## <span style="color:red"> $\Rightarrow$ EXERCICE:   jouer sur les paramètres du réseau (nombre de couches, de neurones dans les couches, d'epochs, ...) et mesurer l'impact sur les performances de classification.</span>